In [1]:
#!/bin/python3

import numpy as np
import keras
from keras.models import Model
from keras.layers import Dense, Input
import tensorflow as tf

import keras_uncertainty
from keras_uncertainty.models import DeepEnsembleRegressor, deep_ensemble_regression_nll_loss

import matplotlib.pyplot as plt



Using TensorFlow backend.
/home/dfki.uni-bremen.de/bmanickavasakan/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/dfki.uni-bremen.de/bmanickavasakan/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/dfki.uni-bremen.de/bmanickavasakan/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint1

In [2]:
def toy_dataset(input):
    output = []

    for inp in input:
        std = 10 if inp > 0 else 2
        out = inp ** 3 + np.random.normal(0, std)
        output.append(out)

    return np.array(output)

def mlp_model():
    inp = Input(shape=(1,))
    x = Dense(10, activation="relu")(inp)
    x = Dense(20, activation="relu")(x)
    x = Dense(30, activation="relu")(x)
    mean = Dense(1, activation="linear")(x)
    var = Dense(1, activation="softplus")(x)

    train_model = Model(inp, mean)
    pred_model = Model(inp, [mean, var])

    train_model.compile(loss=deep_ensemble_regression_nll_loss(var), optimizer="adam")

    return train_model, pred_model



In [7]:
if __name__ == "__main__":
    x_train = np.linspace(-4.0, 4.0, num=1200)
    x_test = np.linspace(-7.0, 7.0, 200)

    y_train = toy_dataset(x_train)
    y_test = toy_dataset(x_test)

    with tf.device('/cpu:0'):
        model = DeepEnsembleRegressor(mlp_model, 5)
        model.fit(x_train, y_train, epochs=200)

    model.save("regression-ens")

    y_pred_mean, y_pred_std = model.predict(x_test)
    y_pred_mean = y_pred_mean.reshape((-1,))
    y_pred_std = y_pred_std.reshape((-1,))

    print("y pred mean shape: {}, y_pred_std shape: {}".format(y_pred_mean.shape, y_pred_std.shape))

    y_pred_up_1 = y_pred_mean + y_pred_std
    y_pred_down_1 = y_pred_mean - y_pred_std

    y_pred_up_2 = y_pred_mean + 2.0 * y_pred_std
    y_pred_down_2 = y_pred_mean - 2.0 * y_pred_std

    y_pred_up_3 = y_pred_mean + 3.0 * y_pred_std
    y_pred_down_3 = y_pred_mean - 3.0 * y_pred_std
    
    plt.plot(x_test, y_test, '.', color=(0, 0.9, 0.0, 0.8), markersize=12, label="Ground truth Points")
    plt.plot(x_test, x_test ** 3, color='red', label="Ground truth 4x**3")

    plt.fill_between(x_test, y_pred_down_3, y_pred_up_3, color=(0, 0, 0.9, 0.2), label="Three Sigma Confidence Interval")
    plt.fill_between(x_test, y_pred_down_2, y_pred_up_2, color=(0, 0, 0.9, 0.5), label="Two Sigma Confidence Interval")
    plt.fill_between(x_test, y_pred_down_1, y_pred_up_1, color=(0, 0, 0.9, 0.7), label="One Sigma Confidence Interval")
        
    plt.legend(loc="upper left")
    plt.title("Deep Ensemble Regression of $x ^ 3 +$ noise with input-dependent noise")

    plt.show()

1200 (200,)
Epoch 1/200
1200/1200 [==============================] - 0s 356us/step - loss: 175.5494
Epoch 2/200
1200/1200 [==============================] - 0s 37us/step - loss: 97.2480
Epoch 3/200
1200/1200 [==============================] - 0s 40us/step - loss: 65.0701
Epoch 4/200
1200/1200 [==============================] - 0s 43us/step - loss: 45.2717
Epoch 5/200
1200/1200 [==============================] - 0s 40us/step - loss: 34.3847
Epoch 6/200
1200/1200 [==============================] - 0s 38us/step - loss: 27.6135
Epoch 7/200
1200/1200 [==============================] - 0s 44us/step - loss: 23.2854
Epoch 8/200
1200/1200 [==============================] - 0s 42us/step - loss: 20.2134
Epoch 9/200
1200/1200 [==============================] - 0s 41us/step - loss: 17.8358
Epoch 10/200
1200/1200 [==============================] - 0s 40us/step - loss: 15.9147
Epoch 11/200
1200/1200 [==============================] - 0s 44us/step - loss: 14.3140
Epoch 12/200
1200/1200 [==============

KeyboardInterrupt: 

In [10]:
print(x_train.shape, x_test.shape)
print(y_train.shape, y_test.shape)



(1200,) (200,)
(1200,) (200,)
